# MAP BIM INFORMATION TO A LAS POINT CLOUD
In this notebook, we select specific elements and classes from an IFC

In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## USER INPUT

In [12]:
## INPUTS
projectPath= os.path.join("D:\\Data\\2023-01 Paestum")
# sessionPath = os.path.join(projectPath,"Research")
#BIM
bimPath=os.path.join(projectPath,"BIM")
ifcPath=os.path.join(projectPath,'BIM','Paestum.ifc')

# bimClasses= {'COLONNA':['COLONNA','PARASTA'],
#             'PILASTRO':'PILASTRO',
#             'TRABEAZIONE':'TRABEAZIONE',
#             'ARCO':'ARCO',
#             'AGGETTO':'aggetto',
#             'FACADE':['CORNICE','MUR']}

transform=np.array([[1.0,0.0, 0.0,  5.3857862609899980e+000], 
                [0.0, 1.0, 0.0, 2.5782303777102851e+002],
                [0.0, 0.0, 1.0 ,-6.0074548600459288e+000],
                [0.0 ,0.0, 0.0, 1.000000000000]]) # -> apply to pcd
    
#PCD
resolution=0.05
distanceThreshold=0.2 #distance theshold for inliers
lasPath=os.path.join(projectPath,'PCD','TEMPIO_all_predicted - Cloud.las')
outputlasPath=os.path.join(projectPath,'PCD','TEMPIO_groundtruth1.las')

#MESH
meshFolderPath=os.path.join(projectPath,'MESH')

# PARSE INPUTS & CREATE NODES

**Fig.**: Images of the BIM section of the colosseum with (a) BIM elements and (b) the point cloud, (c) selected subgroups per model type and (d) segmented point cloud per subgroup.

<img src="../docs/pics/colosseum/columns1.PNG" width = "20%">
<img src="../docs/pics/colosseum/columns2.PNG" width = "25%">

<img src="../docs/pics/colosseum/columns3.PNG" width = "23%">
<img src="../docs/pics/colosseum/columns4.PNG" width = "20%">

## 0.Import the IFC Model.

In [16]:
bimNodes=tl.ifc_to_nodes_multiprocessing(ifcPath,offsetTransform=transform)
# for n in bimNodes:
    # n.cartesianTransform= transform 
    # n.resource.transform(transform)
bimNodes=[n for n in bimNodes if n.resource is not None]
print(f' {str(len(bimNodes))} BIMNodes created!')

 214 BIMNodes created!


In [6]:
{key:value for key, value in bimNodes[2].__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_ifcPath': 'D:\\Data\\2023-01 Paestum\\BIM\\Paestum.ifc',
 '_globalId': '2m5cC6lu9D_9s0K5cLgC0n',
 '_cartesianBounds': array([-10.23262847,  -8.64633704,  -2.47863672,  -0.89234528,
          1.45      ,   6.65      ]),
 '_orientedBounds': array([[ -8.77068506,  -2.59766762,   1.45      ],
        [ -8.77068506,  -2.59766762,   6.65      ],
        [ -8.52730613,  -1.01669331,   1.45      ],
        [-10.35165937,  -2.35428869,   1.45      ],
        [-10.10828044,  -0.77331438,   6.65      ],
        [-10.10828044,  -0.77331438,   1.45      ],
        [-10.35165937,  -2.35428869,   6.65      ],
        [ -8.52730613,  -1.01669331,   6.65      ]]),
 '_orientedBoundingBox': OrientedBoundingBox: center: (-9.43948, -1.68549, 4.05), extent: 5.2, 1.5996, 1.5996),
 '_subject': rdflib.term.URIRef('file:///Colonna_Centrale_Colonna_Dorica_Centrale_142123_2m5cC6lu9D_9s0K5cLgC0n'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': 'Colonna_Centrale:Colonna_Dorica_Centrale:142123',

## 1.Select BIM objects per class based on their name.

In [10]:
classes=[n.objectType for n in bimNodes]
class_set=set(classes)
unique_classes = (list(class_set))
print (unique_classes)
print (len(unique_classes))


['ABACO', 'Colonna_Grande_Centrale:Colonna_Dorica_Grande_Centrale', 'Pilastro_Piccolo:Pilastro_Piccolo', 'TIMPANO', 'Colonna_Centrale:Colonna_Dorica_Centrale', 'Colonna_Superiore_Centrale:Colonna_Dorica_Superiore_Centrale', 'Pilastro_Grande:Pilastro_Grande', 'ECHINO', 'ARCHITRAVE', 'Opening', 'CORNICE', 'Colonna_Esterna:colonna_dorica_esterna', 'Crepidoma:Crepidoma', 'FREGIO', 'Crepidoma_Centrale:Crepidoma_Centrale']
15


In [15]:
joined=gmu.join_geometries([n.resource for n in bimNodes])
o3d.io.write_triangle_mesh(os.path.join(meshFolderPath,f'bimMeshes.obj'),joined) 

True

In [ ]:
no Opening
[garden, Crepidoma, top of Crepidoma, Colonna+Pilastro (maybe seperate these), ECHINO, ABACO,ARCHITRAVE,FREGIO,CORNICE,TIMPANO]

merge pilastro classes, colonna classes

In [7]:
print(bimClasses.values())

dict_values([['COLONNA', 'PARASTA'], 'PILASTRO', 'TRABEAZIONE', 'ARCO', 'aggetto', ['CORNICE', 'MUR']])


In [8]:
# make this exclusive
tcnodeLists=[]
tcgeometries=[] 
for i,c,name in zip(range(len(bimClasses.values())),bimClasses.values(),bimClassNames):
    #select nodes
    c=ut.item_to_list(c)
    tcnodeLists.append([n for n in bimNodes if any(name in n.name for name in c )])
    print(f'{i},  {len(tcnodeLists[i])}, {c} !')
    #combine geometries
    if tcnodeLists[i] is not None:
        tcgeometries.append(gmu.join_geometries([n.resource for n in tcnodeLists[i]]))
        #export geometries     
        o3d.io.write_triangle_mesh(os.path.join(MeshTCFolder,f'{i}_{ut.validate_string(name)}.obj'),tcgeometries[i]) 

0,  65, ['COLONNA', 'PARASTA'] !
1,  75, ['PILASTRO'] !
2,  13, ['TRABEAZIONE'] !
3,  61, ['ARCO'] !
4,  19, ['aggetto'] !
5,  137, ['CORNICE', 'MUR'] !


Create point cloud per mesh geometry.

In [9]:
# ref_clouds= [gmu.mesh_sample_points_uniformly(mesh, resolution=0.1) for mesh in tcgeometries]

(optional) read sample mesh

In [10]:
# mesh=o3d.io.read_triangle_mesh(meshPath)
# k = round(mesh.get_surface_area() * 1000)
# ref_clouds = [mesh.sample_points_uniformly(number_of_points = k, use_triangle_normal=True)]

## 2.Create list of unique materials in the BIMNodes

In [11]:
materials=[m for n in bimNodes for m in n.materials]
materials_set=set(materials)
unique_materials = (list(materials_set))
print (unique_materials)

['Calcestruzzo, gettato in opera', '<Unnamed>', 'CL_00_Calcestruzzo-Generico', 'PT_00_Pietra-Generico', 'PT_10_Pietra_Muratura-XXX-XXX-Secco_Pietrame', 'LT_00_Laterizio-Generico', 'PT_01_Pietra-Travertino', 'BT_00_Bitume-Generico', 'PT_23_Pietra_Rivestimento-XXX-XXX-PietraNaturale_Travertino', 'Sombra escala']


(optionally) save groups of materials as .obj

In [12]:
# make this exclusive
materialNodeLists=[]
materialGeometries=[]
for i,mat in enumerate(unique_materials):
    #select nodes
    materialNodeLists.append([n for n in bimNodes if any(m in mat for m in n.materials )])
    print(f'{i},  {len(materialNodeLists[i])}, {mat} !')
    # combine geometries
    if materialNodeLists[i] is not None:
        materialGeometries.append(gmu.join_geometries([n.resource for n in materialNodeLists[i]]))
        #export geometries     
        o3d.io.write_triangle_mesh(os.path.join(MeshMatFolder,f'{i}_{ut.validate_string(mat)}.obj'),materialGeometries[i])

0,  13, Calcestruzzo, gettato in opera !
1,  241, <Unnamed> !
2,  15, CL_00_Calcestruzzo-Generico !
3,  11, PT_00_Pietra-Generico !
4,  6, PT_10_Pietra_Muratura-XXX-XXX-Secco_Pietrame !
5,  45, LT_00_Laterizio-Generico !
6,  4, PT_01_Pietra-Travertino !
7,  2, BT_00_Bitume-Generico !
8,  2, PT_23_Pietra_Rivestimento-XXX-XXX-PietraNaturale_Travertino !
9,  8, Sombra escala !


## 3.Add class and material indices to bimNodes

In [13]:
for n in bimNodes:
    for i, c in enumerate(bimClasses.values()):
        if any(name in n.name for name in c ):
            n.bimClassField= i 
            break  
        else  :
            n.bimClassField=-1 
        
    for i, mat in enumerate(unique_materials):
        if len(n.materials)>0 and n.materials[0] in mat:
            n.materialField1= i  
            break
        else:
            n.materialField1=-1
    for i, mat in enumerate(unique_materials):
        if len(n.materials)>1 and n.materials[1] in mat:
            n.materialField2= i  
            break
        else:
            n.materialField2=-1        

In [6]:
{key:value for key, value in bimNodes[2].__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_ifcPath': 'D:\\Data\\2023-01 Paestum\\BIM\\Paestum.ifc',
 '_globalId': '2m5cC6lu9D_9s0K5cLgC0W',
 '_cartesianBounds': array([ 0.24627407,  1.83256551, -2.47863672, -0.89234528,  1.45      ,
         6.65      ]),
 '_orientedBounds': array([[ 0.24627407, -0.89234528,  6.65      ],
        [ 0.24627407, -0.89234528,  1.45      ],
        [ 1.83256551, -0.89234528,  6.65      ],
        [ 0.24627407, -2.47863672,  6.65      ],
        [ 1.83256551, -2.47863672,  1.45      ],
        [ 1.83256551, -2.47863672,  6.65      ],
        [ 0.24627407, -2.47863672,  1.45      ],
        [ 1.83256551, -0.89234528,  1.45      ]]),
 '_orientedBoundingBox': OrientedBoundingBox: center: (1.03942, -1.68549, 4.05), extent: 5.2, 1.58629, 1.58629),
 '_subject': rdflib.term.URIRef('file:///Colonna_Centrale_Colonna_Dorica_Centrale_142138_2m5cC6lu9D_9s0K5cLgC0W'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': 'Colonna_Centrale:Colonna_Dorica_Centrale:142138',
 '_timestamp': '2023-02-01T0

Visualize the BIM Classes in different colors.

In [17]:
coloredGeometries=[g.paint_uniform_color(ut.random_color()) for g in materialGeometries if g is not None]

o3d.visualization.draw_geometries(coloredGeometries)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 


# SEGMENT PCD WITH BIM USING LASPY

Read Las data (1.5 min for 110M points, requires 13Gb RAM)

In [20]:
import laspy
las  = laspy.read(lasPath)

Compute nearest neighbors between BIM and point cloud. Fast but operates on single core (10min for 110M points at 0.1m resolution, 13Gb RAM required)

In [138]:
ref_cloud,ref_arr=gmu.create_identity_point_cloud([n.resource for n in bimNodes],resolution=resolution)
query_points=gmu.transform_points( las.xyz,bimTransform)
indices,distances=gmu.compute_nearest_neighbors(query_points,np.asarray(ref_cloud.points))
index=ref_arr[indices]
distances=distances[:,0]

(Optionally): use normals and distanceThreshold to filter it better. (takes twice as long)

In [21]:
ref_cloud,ref_arr=gmu.create_identity_point_cloud([n.resource for n in bimNodes],resolution=resolution,getNormals=True)
query_points,query_normals=gmu.get_points_and_normals(las,transform=bimTransform,getNormals=True)
reference_points,reference_normals=gmu.get_points_and_normals(ref_cloud,getNormals=True)
indices,distances=gmu.compute_nearest_neighbor_with_normal_filtering(query_points,query_normals,reference_points,reference_normals,distanceThreshold=distanceThreshold)
index=ref_arr[indices]

**Fig.**: Nearest neighbor estimation (a) without normal filtering and (b) with normal filtering. Last Figure is the assignment of BIM information in the las point cloud.

<img src="../docs/pics/colosseum/normal_filtering0.PNG" width = "20%">
<img src="../docs/pics/colosseum/normal_filtering1.PNG" width = "22%">
<img src="../docs/pics/colosseum/laspy1.PNG" width = "50%">

Map index to Building techniques and materials

In [22]:
BuildingTechniqueArray=np.zeros(len(las.xyz))
materialsArray1=np.zeros(len(las.xyz))
materialsArray2=np.zeros(len(las.xyz))

for ind in np.unique(index):
    locations=np.where(index ==ind)
    np.put(BuildingTechniqueArray,locations,bimNodes[ind].bimClassField)
    np.put(materialsArray1,locations,bimNodes[ind].materialField1)
    np.put(materialsArray2,locations,bimNodes[ind].materialField2)

Assign building technique, 1st material, 2nd material and distance to the BIM as extra dimensions in the las file. (query points take 4Gb with 110M points so put it in function)

In [23]:
gmu.las_add_extra_dimensions(las,(BuildingTechniqueArray,materialsArray1,materialsArray2,distances),['bimTC','bimMaterial1','bimMaterial2','bimDistance'],['uint8','uint8','uint8','float32'])
print(list(las.point_format.dimension_names))

['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'red', 'green', 'blue', '01 Materiali', '03 D', '02 TC', 'Original cloud index', 'bimTC', 'bimMaterial1', 'bimMaterial2', 'bimDistance']


In [24]:
print(las['bimTC'])
print(las['bimMaterial1'])
print(las['bimMaterial2'])
print(las['bimDistance'])

[1 1 1 ... 0 0 0]
[255 255 255 ...   1   1   1]
[255 255 255 ... 255 255 255]
[0.03154535 0.07160249 0.0433292  ... 0.04762131 0.04273732 0.01054358]


Export las file

In [25]:
las.write(outputlasPath)

**Fig.**: Images of the segmented point cloud with (a) Point cloud enirched with bim class as a feature (b) the ref_clouds from the BIM model, (c) the initial point cloud.

<img src="../docs/pics/colosseum/columns5.PNG" width = "17%">
<img src="../docs/pics/colosseum/columns6.PNG" width = "23%">
<img src="../docs/pics/colosseum/columns7.PNG" width = "23%">

# SEGMENT PCD AS DATAFRAME

1. Without optimization (fast but for small point clouds e.g. 10-20M points)

In [46]:
import time
df = pd.read_csv(csvPath,
    sep= ' ',
    header=0,  
    names=["x","y","z","R", "G", "B", "M", "TC", "Nx", "Ny", "Nz" ])
arr=np.zeros(len(df))
pcd=gmu.dataframe_to_pcd(df)
# pcd.transform(bimTransform)
#compute distance to identityPointCloud   
for i,ref_cloud in enumerate(ref_clouds):
    time
    distances=pcd.compute_point_cloud_distance(ref_cloud)
    #select indices within a distance threshold
    indices=np.where(np.asarray(distances) <= threshold)[0]
    np.put(arr, indices, i)
# assign new column and export df
df = df.assign(className=arr)

# df['class'] = arr.tolist()
df.to_csv(csvPath,mode='a', header=False)

In [ ]:
# for i,ref_cloud,name in zip(range(len(bimClasses.values())),ref_clouds,bimClassNames):


1. Chunked without optimization (slow but memory proof for medium point clouds e.g. 20-100M points)

In [30]:
chunks  = pd.read_csv(csvPath,
    sep= ' ',
    header=0,  
    names=["x","y","z","R", "G", "B", "M", "TC", "Nx", "Ny", "Nz" ],
    chunksize=chunksize,
    iterator=True)
for chunk in chunks: 
    # create integer based array for the classes
    arr=np.zeros(len(chunk))   
    #create point cloud
    pcd=gmu.dataframe_to_pcd(chunk)
    #transform to local coordinate system
    pcd.transform(bimTransform)
    #compute distance to identityPointCloud    
    for i,ref_cloud in enumerate(ref_clouds):
        distances=pcd.compute_point_cloud_distance(ref_cloud)
        #select indices within a distance threshold       
        ind=np.where(np.asarray(distances) <= threshold)[0]
        np.put(arr, indices, i)
    # assign new column and export df
    df['class'] = arr  
    chunk.to_csv(csvPath, mode='a', header=False)

In [ ]:
 
#  # this is some unused code to iteratively write data to csv file
#   test1=chunk.iloc[ind]
#             # #export point clouds
#             with open(os.path.join(ClassPointCloudsFolder,name+'.csv'), "a") as csv:
#                 test1.to_csv(csv,mode='a')
#             print(f'{len(test1)} of {chunksize} exported.')


1. DASK multiprocessing optimization (fast and memory proof for large point clouds e.g. >100M points). Note that this is three times slower for small point clouds due to working spawning, etc.

In [35]:
import dask.dataframe as dd

df = dd.read_csv(csvPath,
                 header=0, 
                sep= ' ')


def create_and_compute_distance(partition, ref_clouds):
    points=partition[partition.columns[:3]].values    
    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))
    for i,ref_cloud,name in zip(range(len(bimClasses.values())),ref_clouds,bimClassNames):
        distances=pcd.compute_point_cloud_distance(ref_cloud)
        #select indices within a distance threshold       
        ind=np.where(np.asarray(distances) <= threshold)[0]
        np.put(arr, indices, i)
    distance = pcd.compute_point_cloud_distance(ref_cloud)
    
    partition['distance_to_ref'] = distance
    return partition

def create_point_cloud_dask(df, ref_cloud):
    point_cloud_partitions = df.map_partitions(create_and_compute_distance, ref_clouds)
    return point_cloud_partitions.compute()

result = create_point_cloud_dask(df, ref_clouds).to_parquet
print(result)

                 //X             Y          Z    R    G   B  01_Materiali  \
0       2.311879e+06  4.640674e+06  32.195537   74   67  58           0.0   
1       2.311879e+06  4.640674e+06  32.184521   90   84  73           0.0   
2       2.311879e+06  4.640674e+06  32.204296  117  102  89           0.0   
3       2.311879e+06  4.640674e+06  32.212200  116  106  91           0.0   
4       2.311879e+06  4.640673e+06  32.135959   79   68  57           0.0   
...              ...           ...        ...  ...  ...  ..           ...   
571611  2.311875e+06  4.640673e+06  35.283402   46   54  49           0.0   
571612  2.311875e+06  4.640673e+06  35.177899   80   80  81           0.0   
571613  2.311875e+06  4.640673e+06  35.307801   44   51  48           0.0   
571614  2.311875e+06  4.640673e+06  35.254101   96  101  98           0.0   
571615  2.311875e+06  4.640673e+06  35.325299   87   88  90           0.0   

        03_D  02_TC  Original_cloud_index        Nx        Ny        Nz  \


In [ ]:
# first dask attempt
# #define a function
# def compute_index(df):
#     xyz=df.iloc[:,[0,1,2]]
#     pcd=o3d.geometry.PointCloud()
#     pcd.points=o3d.utility.Vector3dVector(xyz.to_numpy())
#     # pcd=gmu.dataframe_to_pcd(df)
#     return 10

# #compute function as delayed function
# lazy_results = []
# for chunk in dask_dataframe: #test
#     chunk=chunk.compute()
#     lazy_result = dask.delayed(compute_index)(chunk)
#     lazy_results.append(lazy_result)
# # dask.compute(*lazy_results)
# futures = dask.persist(*lazy_results)  # trigger computation in the background
# client.cluster.scale(10)  # ask for ten 4-thread workers
# results = dask.compute(*futures)
# results[:5]


# results = dask.compute(*futures)
# results[:5]

In [ ]:

#this function does not work because you cannot jointly write in parallel in the same file
# for chunk in dask_dataframe.partitions:    
#     chunk=chunk.compute()
#     # print(chunk[0])
#     # xyz=chunk.get(['Y', 'Z'])

#     pcd=gmu.dataframe_to_pcd(chunk)
#     # xyz=chunk.
#     # print(pcd)
    

#     # pcd=o3d.geometry.PointCloud()
#     # pcd.points=o3d.utility.Vector3dVector(xyz.to_numpy())
#     # #transform to local coordinate system
#     pcd.transform(bimTransform)
#     #compute distance to identityPointCloud    
#     for bimpcd,name in zip(bimPointClouds,bimClassNames):
#         distances=pcd.compute_point_cloud_distance(bimpcd)
#         #remove distances > threshold
#         ind=np.where(np.asarray(distances) <= threshold)[0]
#         #select indices based on closest point        
#         if ind.size >0:
#             test1=chunk.iloc[ind]
#             # #export point clouds
#             with open(os.path.join(ClassPointCloudsFolder,name+'.csv'), "a") as csv:
#                 test1.to_csv(csv,mode='a')
#             print(f'{len(test1)} of {len(chunk)} exported.')


(optional) filter distance calculation based on geometry shape.